In [1]:
#To supress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Dependency for mlxtend
import six
import sys
sys.modules['sklearn.externals.six'] = six

from mlxtend.regressor import StackingCVRegressor

In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
from sklearn import preprocessing

# to build machine learning models
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# to evaluate the models
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

sys.modules['sklearn.externals.six'] = six

In [5]:
final_df = pd.read_csv('final_df.csv')
final_df.head()

,device_id,longitude_y,latitude_y,avg_event,clusterId,gender,age,event_id,dayofweek_num,day,hour,week,is_active,label_id,category
0,-9222956879900150000,113.24,23.19,26,0,M,36,205471,4,6,15,18,0,761,Financial Services
1,-9222956879900150000,113.24,23.19,26,0,M,36,205471,4,6,15,18,0,959,financial
2,-9222956879900150000,113.24,23.19,26,0,M,36,205471,4,6,15,18,0,761,Financial Services
3,-9222956879900150000,113.24,23.19,26,0,M,36,205471,4,6,15,18,0,777,Low liquidity
4,-9222956879900150000,113.24,23.19,26,0,M,36,205471,4,6,15,18,0,959,financial


In [6]:
final_df['gender'] = final_df['gender'].apply(lambda x:1 if x=='M' else 0)

In [17]:
x1 = final_df.drop(["gender","category"] , axis = 1)
y1 = final_df["gender"]

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x1,y1, test_size=0.4, random_state=123, shuffle=True)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((81231, 13), (81231,), (54155, 13), (54155,))

In [19]:
from sklearn import preprocessing

# #scaling data
scaler = preprocessing.StandardScaler()
X_train[['longitude_y','latitude_y','avg_event','clusterId']] = scaler.fit_transform(x_train[['longitude_y','latitude_y','avg_event','clusterId']])
X_test[['longitude_y','latitude_y','avg_event','clusterId']] = scaler.transform(x_test[['longitude_y','latitude_y','avg_event','clusterId']])

X_train.head()

,device_id,longitude_y,latitude_y,avg_event,clusterId,age,event_id,dayofweek_num,day,hour,week,is_active,label_id
105384,5255884550690110000,0.151818,1.578502,-0.677018,-0.052639,22,2925706,4,6,22,18,1,303
85911,2438946640360980000,0.141592,-1.415662,0.139920,0.814872,33,466119,4,6,16,18,1,777
13169,-7296688647213590000,0.542472,0.073405,0.236030,-0.509224,27,3113965,1,3,12,18,0,306
130285,9125896648976320000,0.526110,-0.168836,1.797823,-0.372249,25,67367,3,5,2,18,0,704
82446,2236270255494200000,-0.048622,1.231172,0.572416,0.312629,36,1543467,5,7,16,18,1,704


In [20]:
def evaluate_model_gender(model):
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    print("R2 score : ", r2_score(X_test, Y_pred))
    print("Mean squared error : ", mean_absolute_error(Y_test,Y_pred))
    print("RMSE : ",np.sqrt(mean_squared_error(Y_test, Y_pred)))

In [21]:
from sklearn.linear_model import Ridge

ridge1 = Ridge(alpha=5.0, random_state=1)
evaluate_model_gender(ridge1)

pred_test1 = ridge.predict(X_test)
test_pred_final1 = pd.DataFrame({'age':Y_test.values, 'age_Predicted':pred_test})
test_pred_final1['device_id'] = X_test.index
test_pred_final1.head(20)

R2 score :  0.06636390916940604
Mean squared error :  0.3808134803260728
RMSE :  0.4364146206129801


,age,age_Predicted,device_id
0,1,0.881020,12500
1,1,0.619859,116727
2,0,0.508159,105612
3,0,0.868316,1764
4,1,0.852583,1301
5,1,0.765545,90723
6,0,0.430246,125306
7,1,0.848750,35740
8,1,0.783216,68132
9,1,0.771778,121969


In [22]:
import xgboost as xgb


xgbr1 = xgb.XGBRegressor(n_estimators = 100, learning_rate =0.5, min_child_weight=3,
                               subsample=0.8, max_depth =9, colsample_bytree=0.8,
                               nthread=4, scale_pos_weight=1, seed = 10) 
evaluate_model_gender(xgbr1)

pred_test1 = xgbr.predict(X_test)
test_pred_final1 = pd.DataFrame({'age':Y_test.values, 'age_Predicted':pred_test})
test_pred_final1['device_id'] = X_test.index
test_pred_final1.head(20)

R2 score :  0.9780332566655764
Mean squared error :  0.02899108667135917
RMSE :  0.06694117232651552


,age,age_Predicted,device_id
0,1,1.004380,12500
1,1,1.006185,116727
2,0,-0.031601,105612
3,0,0.009600,1764
4,1,1.081796,1301
5,1,1.020287,90723
6,0,-0.000291,125306
7,1,1.024137,35740
8,1,0.993543,68132
9,1,0.982597,121969


In [23]:
# model built on data with high cardinality for cabin
from sklearn.ensemble import RandomForestRegressor

# call the model
rf_model1 = RandomForestRegressor(max_depth=30, min_samples_leaf=20, n_estimators=15, n_jobs=-1, random_state=42)

evaluate_model_gender(rf_model1)

pred_test1 = rf_model.predict(X_test)
test_pred_final1 = pd.DataFrame({'age':Y_test.values, 'age_Predicted':pred_test})
test_pred_final1['device_id'] = X_test.index
test_pred_final1.head(20)

R2 score :  0.8923490579175041
Mean squared error :  0.06749388127202517
RMSE :  0.14819016902132107


,age,age_Predicted,device_id
0,1,1.000000,12500
1,1,0.980282,116727
2,0,0.072572,105612
3,0,0.000000,1764
4,1,0.638661,1301
5,1,1.000000,90723
6,0,0.000000,125306
7,1,0.987901,35740
8,1,1.000000,68132
9,1,0.970476,121969


In [26]:
from mlxtend.regressor import StackingCVRegressor

sclf1 = StackingCVRegressor(regressors=[ridge, xgbr], 
                          meta_regressor=rf_model)

labels1 = ['Ridge', 'xgbr', 'randomforest', 'Stackingtrg']

sclf1.fit(X_train.values, Y_train.values)
pred_test1 = sclf1.predict(X_test.values)
R_squared1 = r2_score(T_test, pred_test) 
print('R_Squared ',R_squared)
print("MSE ",mean_squared_error(Y_test, pred_test))
print("RMSE : ", np.sqrt(mean_squared_error(Y_test, pred_test)))
      
pred_test1 = sclf.predict(X_test.values)
test_pred_final1 = pd.DataFrame({'age':Y_test.values, 'age_Predicted':pred_test})
test_pred_final1['device_id'] = X_test.index
test_pred_final1.head()

R_Squared  0.9881789510760232
MSE  0.0024114428105379837
RMSE :  0.04910644367634439


,age,age_Predicted,device_id
0,1,1.000000,12500
1,1,1.000000,116727
2,0,0.000000,105612
3,0,0.000000,1764
4,1,0.978314,1301
